**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [2]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act aims to give the next action performed by the agent. During training, this action is either the one that the agent has learnt as to be the best one or a random one. The second case enables the agent to explore new states that it would not have explored otherwise. Therefore ```epsilon``` represents the tradeoff between explorating and exploiting. At test time we don't want to explore anymore so we always choose what we think is the best action.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [3]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        # Actions: 
        #    0 = try to go right and if cannot, go left
        #    1 = try to go left and if cannot, go right
        #    2 = try to go down and if cannot, go up
        #    3 = try to go up and if cannot, go down
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        # Return only what's visible for the rat
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        # No bonus and malus on the same cell
        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        # Nothing on the initial cell
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [4]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 500 # set small when debugging
epochs_test = 100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The array ```position``` is used to show the position of the rat before its action and also to show the accessible points on the island. The array ```board``` holds the positions of the cheese and the poison on the island, once the rat has eaten one of those, it is removed from the board and returned as the reward of the action.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [5]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # Return a random direction
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [5]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [7]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/13.0. Average score (-1.5)
Win/lose count 8.0/12.0. Average score (-2.75)
Win/lose count 11.0/16.0. Average score (-3.5)
Win/lose count 7.5/12.0. Average score (-3.75)
Win/lose count 7.0/12.0. Average score (-4.0)
Final score: -4.0


Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




First of all, using Q definition:
$$Q^{\pi}(s, a) = E_\pi[G_t | S_t=s, A_t=a] \\
    = E_\pi[R_{t+1} + \gamma G_{t+1}|S_t=s, A_t=a] \\
    = \sum_{r, s'} \mathcal{P}(s', r|s,a)(r + \gamma E_\pi[G_{t+1}|S_{t+1}=s']) \\
    = \sum_{r, s'} \mathcal{P}(s', r|s,a)(r + \gamma \sum_{a'}\pi(a'|s')Q^\pi(s', a')) \\
    = \sum_{a', r, s'} \pi(a'|s')\mathcal{P}(s', r|s,a)(r + \gamma Q^\pi(s', a'))$$
Then using the definition of $v^\pi(s) = E_\pi[G_t | S_t=s]$, we get that 
$$v^*(s) = \max_a Q^*(s, a) = \max_a E_{\pi^*}[G_t | S_t=s, A_t=a] \\
    = \max_a E_{\pi^*}[R_{t+1} + \gamma G_{t+1}| S_t=s, A_t=a] \\
    = \max_a \sum_{r, s'} \mathcal{P}(s', r|s,a)(r + \gamma v^*(s')) \\
    = \max_a \sum_{r, s'} \mathcal{P}(s', r|s,a)(r + \gamma \max_{a'}Q^*(s', a'))$$
Hence the relation 
$$ Q^*(s,a) = \sum_{r, s'} \mathcal{P}(s', r|s,a)(r + \gamma \max_{a'}Q^*(s', a'))$$
As we want to estimate $Q^*$, we want to create a function that follows the same properties. That is why the given loss makes sense in our problem.

***
## DQN

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [6]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        # If memory is full, remove first element
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        # Return a random element from memory
        index = np.random.randint(len(self.memory))
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [7]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [8]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.array([s])), axis=1)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            # Retrieve state, next state, action and reward from memory
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            
            # Fill target with predictions of the network
            input_states[i, :, :, :] = s_
            target_q[i, :] = self.model.predict(np.array([s_])).squeeze()
            
            # Update target specific action with cumulative reward
            if game_over_:
                ######## FILL IN
                # Cumulatve reward is only the current reward
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                # Cumulative reward is the current reward + 
                # discount * max over all actions of the estimated Q values of the next state
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict_on_batch(np.array([n_s_])), axis=1).squeeze()
        
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -2, 2)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Reshape((-1,)))
        model.add(Dense(128, activation='relu', input_shape=(5 * 5 * self.n_state,)))
        
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
        self.target_model = keras.models.clone_model(self.model)
        self.target_update = 10
        self.count = 0
        
    def predict_on_batch(batch):
        res = self.target_model.predict_on_batch(batch)
        self.count += 1
        if self.count == self.target_update:
            self.count = 0
            self.target_model = keras.models.clone_model(self.model)
        return res

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.01, epsilon = 0.1, memory_size=10000, batch_size = 16)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train450.mp4'))

Epoch 000/500 | Loss 0.0000 | Win/lose count 0/2.0 (-2.0)
Epoch 001/500 | Loss 0.0022 | Win/lose count 4.5/9.0 (-4.5)
Epoch 002/500 | Loss 0.0022 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/500 | Loss 0.0017 | Win/lose count 3.0/4.0 (-1.0)
Epoch 004/500 | Loss 0.0056 | Win/lose count 3.0/3.0 (0.0)
Epoch 005/500 | Loss 0.0217 | Win/lose count 4.0/6.0 (-2.0)
Epoch 006/500 | Loss 0.0010 | Win/lose count 4.0/5.0 (-1.0)
Epoch 007/500 | Loss 0.0062 | Win/lose count 1.5/1.0 (0.5)
Epoch 008/500 | Loss 0.0108 | Win/lose count 4.0/3.0 (1.0)
Epoch 009/500 | Loss 0.0072 | Win/lose count 2.5/3.0 (-0.5)
Epoch 010/500 | Loss 0.0237 | Win/lose count 7.0/6.0 (1.0)
Epoch 011/500 | Loss 0.0034 | Win/lose count 8.0/4.0 (4.0)
Epoch 012/500 | Loss 0.0020 | Win/lose count 7.0/4.0 (3.0)
Epoch 013/500 | Loss 0.0019 | Win/lose count 2.5/3.0 (-0.5)
Epoch 014/500 | Loss 0.0271 | Win/lose count 4.0/3.0 (1.0)
Epoch 015/500 | Loss 0.0294 | Win/lose count 0.5/1.0 (-0.5)
Epoch 016/500 | Loss 0.0027 | Win/lose count 4.5/

Epoch 139/500 | Loss 0.0042 | Win/lose count 4.5/3.0 (1.5)
Epoch 140/500 | Loss 0.0069 | Win/lose count 7.5/6.0 (1.5)
Epoch 141/500 | Loss 0.0186 | Win/lose count 8.5/4.0 (4.5)
Epoch 142/500 | Loss 0.0027 | Win/lose count 5.5/4.0 (1.5)
Epoch 143/500 | Loss 0.0137 | Win/lose count 8.5/6.0 (2.5)
Epoch 144/500 | Loss 0.0068 | Win/lose count 5.5/7.0 (-1.5)
Epoch 145/500 | Loss 0.0207 | Win/lose count 6.0/3.0 (3.0)
Epoch 146/500 | Loss 0.0085 | Win/lose count 7.5/2.0 (5.5)
Epoch 147/500 | Loss 0.0132 | Win/lose count 5.5/5.0 (0.5)
Epoch 148/500 | Loss 0.0161 | Win/lose count 7.0/1.0 (6.0)
Epoch 149/500 | Loss 0.0161 | Win/lose count 2.5/1.0 (1.5)
Epoch 150/500 | Loss 0.0028 | Win/lose count 9.0/5.0 (4.0)
Epoch 151/500 | Loss 0.0162 | Win/lose count 5.5/4.0 (1.5)
Epoch 152/500 | Loss 0.0032 | Win/lose count 7.0/3.0 (4.0)
Epoch 153/500 | Loss 0.0034 | Win/lose count 4.5/3.0 (1.5)
Epoch 154/500 | Loss 0.0040 | Win/lose count 3.5/2.0 (1.5)
Epoch 155/500 | Loss 0.0052 | Win/lose count 4.0/2.0 (2

Epoch 278/500 | Loss 0.0031 | Win/lose count 4.0/4.0 (0.0)
Epoch 279/500 | Loss 0.0126 | Win/lose count 2.0/0 (2.0)
Epoch 280/500 | Loss 0.0051 | Win/lose count 5.5/4.0 (1.5)
Epoch 281/500 | Loss 0.0014 | Win/lose count 9.5/4.0 (5.5)
Epoch 282/500 | Loss 0.0101 | Win/lose count 6.0/2.0 (4.0)
Epoch 283/500 | Loss 0.0039 | Win/lose count 4.0/1.0 (3.0)
Epoch 284/500 | Loss 0.0095 | Win/lose count 3.5/5.0 (-1.5)
Epoch 285/500 | Loss 0.0114 | Win/lose count 8.0/3.0 (5.0)
Epoch 286/500 | Loss 0.0070 | Win/lose count 5.0/5.0 (0.0)
Epoch 287/500 | Loss 0.0022 | Win/lose count 4.0/3.0 (1.0)
Epoch 288/500 | Loss 0.0072 | Win/lose count 8.0/3.0 (5.0)
Epoch 289/500 | Loss 0.0023 | Win/lose count 3.5/1.0 (2.5)
Epoch 290/500 | Loss 0.0032 | Win/lose count 3.0/0 (3.0)
Epoch 291/500 | Loss 0.0282 | Win/lose count 3.0/1.0 (2.0)
Epoch 292/500 | Loss 0.0040 | Win/lose count 4.0/2.0 (2.0)
Epoch 293/500 | Loss 0.0085 | Win/lose count 7.5/1.0 (6.5)
Epoch 294/500 | Loss 0.0025 | Win/lose count 3.0/2.0 (1.0)


Epoch 417/500 | Loss 0.0354 | Win/lose count 5.5/1.0 (4.5)
Epoch 418/500 | Loss 0.0025 | Win/lose count 8.0/4.0 (4.0)
Epoch 419/500 | Loss 0.0043 | Win/lose count 10.0/5.0 (5.0)
Epoch 420/500 | Loss 0.0064 | Win/lose count 8.0/5.0 (3.0)
Epoch 421/500 | Loss 0.0357 | Win/lose count 3.5/1.0 (2.5)
Epoch 422/500 | Loss 0.0075 | Win/lose count 14.5/4.0 (10.5)
Epoch 423/500 | Loss 0.0040 | Win/lose count 6.0/2.0 (4.0)
Epoch 424/500 | Loss 0.0049 | Win/lose count 8.5/0 (8.5)
Epoch 425/500 | Loss 0.0119 | Win/lose count 5.0/1.0 (4.0)
Epoch 426/500 | Loss 0.0100 | Win/lose count 10.5/4.0 (6.5)
Epoch 427/500 | Loss 0.0050 | Win/lose count 7.5/1.0 (6.5)
Epoch 428/500 | Loss 0.0369 | Win/lose count 1.5/3.0 (-1.5)
Epoch 429/500 | Loss 0.0041 | Win/lose count 6.5/6.0 (0.5)
Epoch 430/500 | Loss 0.0063 | Win/lose count 3.0/1.0 (2.0)
Epoch 431/500 | Loss 0.0354 | Win/lose count 11.0/5.0 (6.0)
Epoch 432/500 | Loss 0.0129 | Win/lose count 6.5/2.0 (4.5)
Epoch 433/500 | Loss 0.0027 | Win/lose count 7.5/1.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [9]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super().__init__(*args, **kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(24, 3, activation='relu', padding='same', data_format='channels_last', input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(48, 2, activation='relu', data_format='channels_last'))
        model.add(Reshape((-1,)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=10000, batch_size = 16)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train450.mp4'))

Epoch 000/500 | Loss 0.0109 | Win/lose count 9.0/12.0 (-3.0)
Epoch 001/500 | Loss 0.0033 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/500 | Loss 0.0022 | Win/lose count 3.0/1.0 (2.0)
Epoch 003/500 | Loss 0.0020 | Win/lose count 1.0/4.0 (-3.0)
Epoch 004/500 | Loss 0.0026 | Win/lose count 7.5/3.0 (4.5)
Epoch 005/500 | Loss 0.0454 | Win/lose count 7.5/2.0 (5.5)
Epoch 006/500 | Loss 0.0042 | Win/lose count 9.5/1.0 (8.5)
Epoch 007/500 | Loss 0.0015 | Win/lose count 7.0/3.0 (4.0)
Epoch 008/500 | Loss 0.0128 | Win/lose count 9.0/1.0 (8.0)
Epoch 009/500 | Loss 0.0011 | Win/lose count 4.5/4.0 (0.5)
Epoch 010/500 | Loss 0.0007 | Win/lose count 12.0/2.0 (10.0)
Epoch 011/500 | Loss 0.0007 | Win/lose count 5.0/4.0 (1.0)
Epoch 012/500 | Loss 0.0030 | Win/lose count 13.5/2.0 (11.5)
Epoch 013/500 | Loss 0.0018 | Win/lose count 3.0/2.0 (1.0)
Epoch 014/500 | Loss 0.0007 | Win/lose count 16.0/2.0 (14.0)
Epoch 015/500 | Loss 0.0056 | Win/lose count 9.5/1.0 (8.5)
Epoch 016/500 | Loss 0.0476 | Win/lose count 1

Epoch 137/500 | Loss 0.0011 | Win/lose count 19.0/3.0 (16.0)
Epoch 138/500 | Loss 0.0017 | Win/lose count 21.0/4.0 (17.0)
Epoch 139/500 | Loss 0.0022 | Win/lose count 20.5/1.0 (19.5)
Epoch 140/500 | Loss 0.0443 | Win/lose count 9.5/0 (9.5)
Epoch 141/500 | Loss 0.0007 | Win/lose count 17.0/1.0 (16.0)
Epoch 142/500 | Loss 0.0003 | Win/lose count 22.0/1.0 (21.0)
Epoch 143/500 | Loss 0.0017 | Win/lose count 16.5/2.0 (14.5)
Epoch 144/500 | Loss 0.0013 | Win/lose count 19.0/4.0 (15.0)
Epoch 145/500 | Loss 0.0002 | Win/lose count 14.0/6.0 (8.0)
Epoch 146/500 | Loss 0.0445 | Win/lose count 10.5/2.0 (8.5)
Epoch 147/500 | Loss 0.0007 | Win/lose count 14.0/3.0 (11.0)
Epoch 148/500 | Loss 0.0019 | Win/lose count 15.5/4.0 (11.5)
Epoch 149/500 | Loss 0.0016 | Win/lose count 21.0/2.0 (19.0)
Epoch 150/500 | Loss 0.0014 | Win/lose count 9.0/2.0 (7.0)
Epoch 151/500 | Loss 0.0016 | Win/lose count 14.5/3.0 (11.5)
Epoch 152/500 | Loss 0.0012 | Win/lose count 26.5/3.0 (23.5)
Epoch 153/500 | Loss 0.0012 | Wi

Epoch 273/500 | Loss 0.0005 | Win/lose count 21.0/1.0 (20.0)
Epoch 274/500 | Loss 0.0026 | Win/lose count 21.0/3.0 (18.0)
Epoch 275/500 | Loss 0.0011 | Win/lose count 25.5/3.0 (22.5)
Epoch 276/500 | Loss 0.0013 | Win/lose count 23.0/2.0 (21.0)
Epoch 277/500 | Loss 0.0005 | Win/lose count 20.5/4.0 (16.5)
Epoch 278/500 | Loss 0.0407 | Win/lose count 22.0/3.0 (19.0)
Epoch 279/500 | Loss 0.0011 | Win/lose count 27.0/2.0 (25.0)
Epoch 280/500 | Loss 0.0007 | Win/lose count 19.0/4.0 (15.0)
Epoch 281/500 | Loss 0.0017 | Win/lose count 19.5/5.0 (14.5)
Epoch 282/500 | Loss 0.0007 | Win/lose count 13.5/1.0 (12.5)
Epoch 283/500 | Loss 0.0013 | Win/lose count 12.5/3.0 (9.5)
Epoch 284/500 | Loss 0.0009 | Win/lose count 23.0/2.0 (21.0)
Epoch 285/500 | Loss 0.0009 | Win/lose count 23.5/4.0 (19.5)
Epoch 286/500 | Loss 0.0010 | Win/lose count 18.5/2.0 (16.5)
Epoch 287/500 | Loss 0.0015 | Win/lose count 12.5/4.0 (8.5)
Epoch 288/500 | Loss 0.0007 | Win/lose count 11.5/1.0 (10.5)
Epoch 289/500 | Loss 0.048

Epoch 408/500 | Loss 0.0007 | Win/lose count 19.0/4.0 (15.0)
Epoch 409/500 | Loss 0.0007 | Win/lose count 18.0/4.0 (14.0)
Epoch 410/500 | Loss 0.0010 | Win/lose count 9.0/4.0 (5.0)
Epoch 411/500 | Loss 0.0010 | Win/lose count 24.0/2.0 (22.0)
Epoch 412/500 | Loss 0.0009 | Win/lose count 22.5/4.0 (18.5)
Epoch 413/500 | Loss 0.0016 | Win/lose count 11.0/3.0 (8.0)
Epoch 414/500 | Loss 0.0020 | Win/lose count 26.5/3.0 (23.5)
Epoch 415/500 | Loss 0.0009 | Win/lose count 22.5/3.0 (19.5)
Epoch 416/500 | Loss 0.0011 | Win/lose count 21.0/2.0 (19.0)
Epoch 417/500 | Loss 0.0012 | Win/lose count 10.5/5.0 (5.5)
Epoch 418/500 | Loss 0.0010 | Win/lose count 17.5/3.0 (14.5)
Epoch 419/500 | Loss 0.0021 | Win/lose count 25.0/3.0 (22.0)
Epoch 420/500 | Loss 0.0005 | Win/lose count 21.0/3.0 (18.0)
Epoch 421/500 | Loss 0.0007 | Win/lose count 26.0/5.0 (21.0)
Epoch 422/500 | Loss 0.0012 | Win/lose count 21.0/2.0 (19.0)
Epoch 423/500 | Loss 0.0021 | Win/lose count 16.0/3.0 (13.0)
Epoch 424/500 | Loss 0.0372 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [10]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 11.5/1.0. Average score (10.5)
Win/lose count 17.5/4.0. Average score (12.0)
Win/lose count 21.0/6.0. Average score (13.0)
Win/lose count 21.5/6.0. Average score (13.625)
Win/lose count 20.5/5.0. Average score (14.0)
Win/lose count 23.0/1.0. Average score (15.333333333333334)
Win/lose count 20.0/2.0. Average score (15.714285714285714)
Win/lose count 25.0/0. Average score (16.875)
Win/lose count 18.0/4.0. Average score (16.555555555555557)
Win/lose count 21.5/3.0. Average score (16.75)
Win/lose count 18.5/2.0. Average score (16.727272727272727)
Win/lose count 21.0/1.0. Average score (17.0)
Win/lose count 18.5/3.0. Average score (16.884615384615383)
Win/lose count 17.0/1.0. Average score (16.821428571428573)
Win/lose count 14.5/3.0. Average score (16.466666666666665)
Win/lose count 11.0/2.0. Average score (16.0)
Win/lose count 24.0/5.0. Average score (16.176470588235293)
Win/lose count 14.0/1.0. Average score (16.0)
Win/lose count 19.5/3.0. Average score (1

Win/lose count 8.5/1.0. Average score (3.8333333333333335)
Win/lose count 9.0/3.0. Average score (3.877551020408163)
Win/lose count 16.5/6.0. Average score (4.01)
Win/lose count 10.0/2.0. Average score (4.088235294117647)
Win/lose count 15.0/4.0. Average score (4.221153846153846)
Win/lose count 6.5/1.0. Average score (4.245283018867925)
Win/lose count 3.5/3.0. Average score (4.175925925925926)
Win/lose count 8.0/1.0. Average score (4.2272727272727275)
Win/lose count 12.5/4.0. Average score (4.303571428571429)
Win/lose count 4.5/4.0. Average score (4.2368421052631575)
Win/lose count 12.5/2.0. Average score (4.344827586206897)
Win/lose count 20.0/3.0. Average score (4.559322033898305)
Win/lose count 10.0/0. Average score (4.65)
Win/lose count 10.5/3.0. Average score (4.69672131147541)
Win/lose count 6.0/1.0. Average score (4.701612903225806)
Win/lose count 14.0/2.0. Average score (4.817460317460317)
Win/lose count 10.5/7.0. Average score (4.796875)
Win/lose count 4.0/2.0. Average score (

In [11]:
HTML(display_videos('cnn_test10.mp4'))

In [12]:
HTML(display_videos('fc_test10.mp4'))

We see that our agent often gets stuck oscillating between two states. This is probably because it gets in a state where it can't find an interesting move anymore, therefore it tries an action that does not allow it to get to a more interesting state.
Changing the temperature does not seem to drastically change the behaviour of our agent. By testing with a temperature of 0.1, 0.2, 0.3 and 0.5 we see that the average score of our agent is between 60 and 70% of the expected bonus on the island.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [13]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    eps = 0.9

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        # At each epoch we decrease epsilon
        agent.set_epsilon(eps)
        eps = max(0.9 * eps, 0.05)

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super().__init__(grid_size=grid_size, max_time=max_time,temperature=temperature)
        # Add a malus on already visited positions
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        
    def act(self, action, train=False):
        state, reward, game_over = super().act(action)
        if train:
            # Decrease reward by the malus value during training
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = min(self.malus_position[self.x, self.y] + 0.1, 0.3)
        
        # Add malus to the state
        state = np.concatenate((self.malus_position[self.x - 2:self.x + 3, self.y - 2:self.y + 3].reshape(5, 5, 1),
                                state), axis=2)
        return state, reward, game_over
        
    def reset(self):
        state = super().reset()
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1
        state = np.concatenate((self.malus_position[self.x - 2:self.x + 3, self.y - 2:self.y + 3].reshape(5, 5, 1),
                                state), axis=2)
        return state

In [14]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=10000, batch_size = 16, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore450.mp4'))

Epoch 000/500 | Loss 0.0272 | Win/lose count 10.5/47.09999999999999 (-36.59999999999999)
Epoch 001/500 | Loss 0.0136 | Win/lose count 10.0/43.099999999999945 (-33.099999999999945)
Epoch 002/500 | Loss 0.0139 | Win/lose count 13.5/38.19999999999999 (-24.69999999999999)
Epoch 003/500 | Loss 0.0255 | Win/lose count 12.0/37.99999999999998 (-25.99999999999998)
Epoch 004/500 | Loss 0.0216 | Win/lose count 15.0/34.100000000000044 (-19.100000000000044)
Epoch 005/500 | Loss 0.0050 | Win/lose count 24.0/30.00000000000004 (-6.000000000000039)
Epoch 006/500 | Loss 0.0068 | Win/lose count 15.0/30.000000000000018 (-15.000000000000018)
Epoch 007/500 | Loss 0.0023 | Win/lose count 18.0/29.200000000000024 (-11.200000000000024)
Epoch 008/500 | Loss 0.0064 | Win/lose count 23.0/24.100000000000016 (-1.1000000000000156)
Epoch 009/500 | Loss 0.0043 | Win/lose count 23.0/19.999999999999993 (3.000000000000007)
Epoch 010/500 | Loss 0.0057 | Win/lose count 16.5/28.40000000000001 (-11.90000000000001)
Epoch 011/5

Epoch 092/500 | Loss 0.0176 | Win/lose count 26.0/14.599999999999982 (11.400000000000018)
Epoch 093/500 | Loss 0.0233 | Win/lose count 22.0/17.599999999999994 (4.400000000000006)
Epoch 094/500 | Loss 0.0207 | Win/lose count 23.0/10.599999999999987 (12.400000000000013)
Epoch 095/500 | Loss 0.0179 | Win/lose count 24.0/19.89999999999998 (4.100000000000019)
Epoch 096/500 | Loss 0.0072 | Win/lose count 18.5/20.6 (-2.1000000000000014)
Epoch 097/500 | Loss 0.0076 | Win/lose count 27.5/13.599999999999985 (13.900000000000015)
Epoch 098/500 | Loss 0.0250 | Win/lose count 24.5/15.699999999999987 (8.800000000000013)
Epoch 099/500 | Loss 0.0192 | Win/lose count 21.0/15.19999999999998 (5.80000000000002)
Epoch 100/500 | Loss 0.0199 | Win/lose count 24.5/17.19999999999999 (7.300000000000011)
Epoch 101/500 | Loss 0.0218 | Win/lose count 25.5/11.799999999999986 (13.700000000000014)
Epoch 102/500 | Loss 0.0685 | Win/lose count 25.5/18.199999999999992 (7.300000000000008)
Epoch 103/500 | Loss 0.0156 | Win

Epoch 186/500 | Loss 0.0272 | Win/lose count 27.0/15.59999999999998 (11.40000000000002)
Epoch 187/500 | Loss 0.0176 | Win/lose count 22.5/19.900000000000002 (2.599999999999998)
Epoch 188/500 | Loss 0.0109 | Win/lose count 28.0/14.899999999999988 (13.100000000000012)
Epoch 189/500 | Loss 0.0320 | Win/lose count 21.5/18.700000000000003 (2.799999999999997)
Epoch 190/500 | Loss 0.0195 | Win/lose count 22.5/11.199999999999983 (11.300000000000017)
Epoch 191/500 | Loss 0.0158 | Win/lose count 22.5/15.099999999999978 (7.400000000000022)
Epoch 192/500 | Loss 0.0292 | Win/lose count 25.5/20.69999999999999 (4.800000000000011)
Epoch 193/500 | Loss 0.0116 | Win/lose count 24.0/20.39999999999999 (3.6000000000000085)
Epoch 194/500 | Loss 0.0221 | Win/lose count 25.5/17.900000000000013 (7.599999999999987)
Epoch 195/500 | Loss 0.0378 | Win/lose count 22.5/12.499999999999979 (10.000000000000021)
Epoch 196/500 | Loss 0.0108 | Win/lose count 21.0/15.499999999999975 (5.500000000000025)
Epoch 197/500 | Loss

Epoch 278/500 | Loss 0.0481 | Win/lose count 25.5/15.899999999999993 (9.600000000000007)
Epoch 279/500 | Loss 0.0218 | Win/lose count 27.5/8.79999999999999 (18.70000000000001)
Epoch 280/500 | Loss 0.0246 | Win/lose count 25.0/11.99999999999998 (13.00000000000002)
Epoch 281/500 | Loss 0.0289 | Win/lose count 23.0/10.499999999999986 (12.500000000000014)
Epoch 282/500 | Loss 0.0300 | Win/lose count 20.0/15.099999999999984 (4.900000000000016)
Epoch 283/500 | Loss 0.0157 | Win/lose count 27.0/10.599999999999984 (16.400000000000016)
Epoch 284/500 | Loss 0.0180 | Win/lose count 20.5/16.199999999999974 (4.300000000000026)
Epoch 285/500 | Loss 0.0157 | Win/lose count 26.5/17.499999999999982 (9.000000000000018)
Epoch 286/500 | Loss 0.0631 | Win/lose count 20.0/16.899999999999984 (3.1000000000000156)
Epoch 287/500 | Loss 0.0512 | Win/lose count 22.5/16.49999999999998 (6.000000000000021)
Epoch 288/500 | Loss 0.0271 | Win/lose count 21.0/14.39999999999998 (6.600000000000019)
Epoch 289/500 | Loss 0.

Epoch 371/500 | Loss 0.0296 | Win/lose count 22.5/15.799999999999981 (6.700000000000019)
Epoch 372/500 | Loss 0.0199 | Win/lose count 23.5/12.299999999999986 (11.200000000000014)
Epoch 373/500 | Loss 0.0205 | Win/lose count 18.5/15.099999999999975 (3.4000000000000252)
Epoch 374/500 | Loss 0.0302 | Win/lose count 25.0/13.79999999999998 (11.20000000000002)
Epoch 375/500 | Loss 0.0155 | Win/lose count 30.0/14.399999999999983 (15.600000000000017)
Epoch 376/500 | Loss 0.0339 | Win/lose count 25.5/13.499999999999977 (12.000000000000023)
Epoch 377/500 | Loss 0.0056 | Win/lose count 22.0/14.799999999999988 (7.200000000000012)
Epoch 378/500 | Loss 0.0290 | Win/lose count 21.0/14.29999999999999 (6.70000000000001)
Epoch 379/500 | Loss 0.0126 | Win/lose count 23.5/14.499999999999982 (9.000000000000018)
Epoch 380/500 | Loss 0.0176 | Win/lose count 19.5/18.199999999999978 (1.300000000000022)
Epoch 381/500 | Loss 0.0183 | Win/lose count 20.0/16.099999999999973 (3.900000000000027)
Epoch 382/500 | Loss

Epoch 463/500 | Loss 0.0133 | Win/lose count 26.5/9.99999999999999 (16.50000000000001)
Epoch 464/500 | Loss 0.0155 | Win/lose count 28.0/16.299999999999976 (11.700000000000024)
Epoch 465/500 | Loss 0.0428 | Win/lose count 21.0/17.69999999999998 (3.3000000000000185)
Epoch 466/500 | Loss 0.0789 | Win/lose count 24.5/10.699999999999987 (13.800000000000013)
Epoch 467/500 | Loss 0.0140 | Win/lose count 27.0/13.399999999999984 (13.600000000000016)
Epoch 468/500 | Loss 0.0200 | Win/lose count 23.0/11.399999999999975 (11.600000000000025)
Epoch 469/500 | Loss 0.0191 | Win/lose count 27.0/18.299999999999983 (8.700000000000017)
Epoch 470/500 | Loss 0.0144 | Win/lose count 24.0/8.399999999999993 (15.600000000000007)
Epoch 471/500 | Loss 0.0195 | Win/lose count 21.5/15.299999999999978 (6.200000000000022)
Epoch 472/500 | Loss 0.0248 | Win/lose count 21.5/12.499999999999977 (9.000000000000023)
Epoch 473/500 | Loss 0.0239 | Win/lose count 25.0/11.599999999999984 (13.400000000000016)
Epoch 474/500 | Lo

In [15]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 28.5/7.0. Average score (21.5)
Win/lose count 15.0/7.0. Average score (14.75)
Win/lose count 26.0/2.0. Average score (17.833333333333332)
Win/lose count 24.0/3.0. Average score (18.625)
Win/lose count 23.5/3.0. Average score (19.0)
Win/lose count 27.0/4.0. Average score (19.666666666666668)
Win/lose count 24.5/1.0. Average score (20.214285714285715)
Win/lose count 22.5/5.0. Average score (19.875)
Win/lose count 21.5/2.0. Average score (19.833333333333332)
Win/lose count 25.5/2.0. Average score (20.2)
Win/lose count 23.5/2.0. Average score (20.318181818181817)
Win/lose count 25.5/5.0. Average score (20.333333333333332)
Win/lose count 24.5/1.0. Average score (20.576923076923077)
Win/lose count 23.0/6.0. Average score (20.321428571428573)
Win/lose count 25.5/3.0. Average score (20.466666666666665)
Win/lose count 20.5/2.0. Average score (20.34375)
Win/lose count 28.0/4.0. Average score (20.558823529411764)
Win/lose count 23.0/1.0. Average score (20.63888888888889)
Win/lose c

We see that our model performs better than before. In fact, it is almost performingly the best it can as with a temperature of 0.3 and a size of 13, the average number of possible points to earn is 25.35. However, all bonus points may not be reachable without crossing a point containing a malus. So the optimal result should be a bit lower than 25.35.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***